In [1]:
import os

In [2]:
%pwd

'c:\\Users\\jhasu\\OneDrive\\Desktop\\chickenclassifier\\chickendisease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\jhasu\\OneDrive\\Desktop\\chickenclassifier\\chickendisease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
import tensorflow as tf

In [7]:
model= tf.keras.models.load_model("artifacts/training/model.keras")

In [8]:
@dataclass(frozen=True)
class Evaluationconfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [12]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directory,save_jason

In [10]:
class ConfigurationManager:

    def __init__(self,
     config_filepath=CONFIG_FILE_PATH,
     param_filepath=PARAM_FILE_PATH):
     self.config = read_yaml(config_filepath)
     self.param = read_yaml(param_filepath)

     create_directory([self.config.artifacts_root])

    def get_evaluation_configuration(self)->Evaluationconfig:
        config=self.config.data_ingestion
        create_directory([config.root_dir])
        evaluation_config= Evaluationconfig(
           path_of_model= Path("artifacts/training/model.keras"),
           training_data= Path("artifacts/data_ingestion/Chicken-fecal-images"),
           all_params= self.param,
           params_image_size=self.param.IMAGE_SIZE,
           params_batch_size=self.param.BATCH_SIZE,
            )

        return evaluation_config

In [11]:
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, evaluation_config: Evaluationconfig):
        self.config= evaluation_config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1/255,
            validation_split= 0.30
        )

        dataflow_kwargs = dict(
            target_size= self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs)
        

    @staticmethod
    def load_model(path: Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
        
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores= {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_jason(path= Path("score.jason"), data= scores)



In [14]:
try:
    config_manager = ConfigurationManager()
    evaluation_config = config_manager.get_evaluation_configuration()
    evaluation = Evaluation(evaluation_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

[2025-01-16 18:36:42,679: INFO : common : yaml file: config\config.yaml loaded successfully ]
[2025-01-16 18:36:42,694: INFO : common : yaml file: params.yaml loaded successfully ]
[2025-01-16 18:36:42,702: INFO : common : Directory: artifacts created successfully ]
[2025-01-16 18:36:42,708: INFO : common : Directory: artifacts/data_ingestion created successfully ]
Found 116 images belonging to 2 classes.


c:\Users\jhasu\anaconda3\envs\cenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.6616 - loss: 1.7543
[2025-01-16 18:36:57,293: INFO : common : Jason file: score.jason saved successfully ]
